<a href="https://colab.research.google.com/github/hjhgo/deep_learning_demo/blob/master/minst_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

sample_data


In [2]:
!rm m*

rm: cannot remove 'm*': No such file or directory


In [3]:
!rm checkpoint

rm: cannot remove 'checkpoint': No such file or directory


In [0]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.layers as layers


def model_fn(features, labels, mode, params):
    def get_input(features,labels):
        input_x = tf.expand_dims(features,axis=-1)
        input_y = tf.squeeze(labels)

        return input_x,input_y

    def conv_net(x):
        initializer = tf.random_normal_initializer(stddev=0.01)
        with tf.name_scope("conv1"):
            filter = tf.get_variable("filter-%s" % 32, [5, 5, 1, 32],
                                     initializer=initializer)

            x = tf.nn.conv2d(x, filter=filter, strides=[1, 1, 1, 1], padding="SAME")
            x = tf.nn.max_pool(x, ksize=(1, 2, 2, 1), strides=[1, 2, 2, 1], padding='VALID')
            if mode==tf.estimator.ModeKeys.TRAIN:
                with tf.name_scope("drop_out"):
                    x = tf.nn.dropout(x, keep_prob=params.drop_out)
        
        with tf.name_scope("conv2"):
            # conv
            filter = tf.get_variable("filter-%s" % 64, [5, 5, 32, 64],
                                     initializer=initializer)
            x = tf.nn.conv2d(x, filter=filter, strides=[1, 1, 1, 1], padding="SAME")
            x = tf.nn.max_pool(x, ksize=(1, 2, 2, 1), strides=[1, 2, 2, 1], padding='VALID')
            if mode==tf.estimator.ModeKeys.TRAIN:
                with tf.name_scope("drop_out"):
                    x = tf.nn.dropout(x, keep_prob=params.drop_out)

        with tf.name_scope("conv3"):
            # conv
            filter = tf.get_variable("filter-%s" % 258, [5, 5, 64, 258],
                                     initializer=initializer)
            x = tf.nn.conv2d(x, filter=filter, strides=[1, 1, 1, 1], padding="SAME")
            x = tf.nn.max_pool(x, ksize=(1, 2, 2, 1), strides=[1, 2, 2, 1], padding='VALID')
            if mode==tf.estimator.ModeKeys.TRAIN:
                with tf.name_scope("drop_out"):
                    x = tf.nn.dropout(x, keep_prob=params.drop_out)            
       
        return x

    def fullconnected(x):
        if mode==tf.estimator.ModeKeys.TRAIN:
            with tf.name_scope("drop_out"):
                x = tf.nn.dropout(x, keep_prob=params.drop_out)
                
        with tf.name_scope("full_connect1"):
            x = tf.layers.dense(x, units=params.hidden_size, activation="elu")

        with tf.name_scope("full_connect2"):
            logist = tf.layers.dense(x, units=params.num_class)
        return logist

    x , y = get_input(features, labels)
    x = conv_net(x)
    size = 1
    for i in x.get_shape().as_list()[1:]:
        size*=i
    x = tf.reshape(x,shape=[-1,size])
    logits = fullconnected(x)
    prediction = tf.argmax(logits,axis=1)
    cross_entropy = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits))
    train_op= layers.optimize_loss(loss=cross_entropy,
                                   global_step=tf.train.get_global_step(),
                                   learning_rate=params.learning_rate,
                                   optimizer="Adam",
                                   clip_gradients=params.clip_max,
                                   )
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions={"logits":logits,"prediction":prediction},
                                      loss=cross_entropy,
                                      train_op=train_op,
                                      eval_metric_ops={"accuracy":tf.metrics.accuracy(labels,prediction)})


# import tensorflow.contrib.training as train
# train.HParams

def create_estimator_and_spec():
    model_param =tf.contrib.training.HParams(
                num_class = 10,
                hidden_size = 256,
                clip_max = 5.0,
                drop_out = 0.7,
                learning_rate = 0.01,
                batch_size = 128
    )
    run_config = tf.estimator.RunConfig(
        model_dir="./",
        save_checkpoints_secs=300,
        save_summary_steps=100)

    estimator = tf.estimator.Estimator(
        model_fn = model_fn,
        config = run_config,
        params=model_param
    )

    return estimator


if __name__ == '__main__':
    estimator = create_estimator_and_spec()
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    train_input_fn = tf.estimator.inputs.numpy_input_fn(x=x_train.astype(np.float32),y=y_train.astype(np.int32),shuffle=True,batch_size=128,num_epochs=10)
    test_input_fn = tf.estimator.inputs.numpy_input_fn(x=x_test.astype(np.float32),y=y_test.astype(np.int32),shuffle=False,batch_size=128,num_epochs=1)

    # estimator.train(input_fn=train_input_fn)
    train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn)
    evel_spec = tf.estimator.EvalSpec(input_fn=test_input_fn)
#    estimator.train(input_fn=train_input_fn)
    tf.estimator.train_and_evaluate(estimator=estimator,train_spec=train_spec,eval_spec=evel_spec)






INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7f8e580ef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will hap